# Set Up

In [5]:
%matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import bepy as be
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, NMF, DictionaryLearning
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [3]:
gridSize = 10
sampName = 'M1B'
path = 'D:\\Dropbox (GaTech)\\Dropbox (GaTech)\\ME-DboxMgmt-Bassiri-Gharb\\BE_Active\\Lee\\pt40_024\\10-07-2019\\Measurements\\'
samp = be.Sample(path, gridSize, adjustphase=False)

C:\ProgramData\Anaconda3\lib\site-packages\bepy\sample.py:30: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  self.addentiresample(path, gridSize, adjustphase)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Basic Plots

## BE Scan Plots

Scan measurements are stored in the ```samp.linemeasurements``` object. Which is a dictionary, so the individual scans can be accessed with  ```samp.linemeasurements['name']```.
The below cell shoud list the name's of the line measurements currently loaded.

In [4]:
print("The scan (or line) measurement are: ", list(samp.linemeasurements.keys()))

The scan (or line) measurement are:  ['Vert_pre']


You can call ```.plot()``` on any measurement to generate a standard plot. For scan measurements, the best way to call ```plot``` is as follows:
    
```python
samp.linemeasurements['Vert'].plot(lims=lims, plotgroup=1)
```

Assuming you used the standard BEline256.csv wavespec, ```plotgroup=1``` denotes that you want to plot the main image area and ```plotgroup=0``` denotes the overscan areas.
```lims``` is a matrix with the plot limits for each variable, structured as such:
```python
[[Amin, Phmin, fmin, Qmin], [Amax,Phmax, fmax, Qmax]]
```
Amin and Amax are the minimum and maximum plot values for the amplitude. Phmin and Phmax are the minimum and maximum plot values for the phase. fmin and fmax are the minimum and maximum plot values for the resonance. Qmin and Qmax are the minimum and maximum plot values for the Q.

In [3]:
# Select Limits
lims = np.array([[0,0,0.395,0],[2000,360,0.400,200]])

#Plot
samp.linemeasurements['Vert_pre'].plot(lims=lims,plotgroup=1)

NameError: name 'samp' is not defined

To maintain consistent image size, you can use get_size_inches to get the size of the figure above in inches. Then you can use set_size_inches to set all of the subsequent figure sizes. Any matplotlib functionality such as this is compatible with the generated plots

In [ ]:
#Get reference to current active figure
fig = plt.gcf()

#Get active figure size
fig.get_size_inches()

#Set active figure size
fig.set_size_inches(18.0, 6.0)

## SSPFM Plot

All grid based measurements (SSPFM, relaxation, etc.) are stored in the ```samp.gridmeasurements``` object. Which is a dictionary, so the individual measurements can be accessed with  ```samp.gridmeasurements['name']```

In [ ]:
print("The grid measurements are: ", list(samp._gridmeasurements.keys()))

You can call ```.plot()``` on any measurement to generate a standard plot. For SSPFM measurements, the best way to call ```plot``` is as follows:
    
```python
samp._gridmeasurements['SSPFM'].plot(pointNum=500)
```

Here pointNum is the desired point number (0 to total number of measured points minus one, e.g. for a 50x50 square, 0-2499)

In [ ]:
samp.gridmeasurements['TEST'].plot(pointNum=None)

fig = plt.gcf()
fig.set_size_inches(18.0, 6.5)

Imslice below tries to reshape the data into the grid (e.g. 2500 -> 50x50). If you are missing some of the 2500 points, this reshaping will fail. But you can just insert the missing points as all nans to fix this

Imslice:

In [ ]:
lims = np.array([[0,0,395,0],[1500,360,410,500]])

samp.gridmeasurements['RT_SSPFM_10VP'].imslice(variables=None, sliceNum=150, InOut=0.0, plotgroup=0, saveName=None, limits=lims.T)

fig = plt.gcf()
fig.set_size_inches(10.0, 10.0)

## Relaxation Plot

The above comments on plotting applies to any gridmeasurement object. However, if a particular gridmeasurement object is a "Relaxation-type" measurement, ```.plot()``` will automatically plot the data against _time_. Conversly, as seen in the above cells the "SSPFM-type" gridmeasurement plot against voltage by default.

Further, all gridmeasurement object plotters accept arguments ```InOut``` and ```insert```, as demonstrated below. ```InOut``` allows the user to select whether the in-field data (```InOut=1```) or the out-of-field data (```InOut=0```) is plotted. ```insert``` allows the user to specify what the removed data is replaced with. E.g.```InOut=1``` and ```insert=np.inf``` will plot the _in-field_ data with the _out-of-field_ data replaced with infs.

In [ ]:
samp.gridmeasurements['Relax'].plot(pointNum=None, InOut=1.0, insert=np.inf)

# Checking Gridmeasurements for Bad Fits

In [ ]:
stack = ['Q']
inout = None # 0 for out of field, 1 for in-field
meas = 'RT_SSPFM_10VN'

stack_len = len(stack)
test_var = stack[0]
num_points, t_length = samp.gridmeasurements[meas].GetDataSubset(stack=[test_var], inout=inout).shape

bad_dict = {}

for var in stack:
    var_data = samp.gridmeasurements[meas].GetDataSubset(stack=[var],inout=inout)
    chirps = var_data.columns.get_level_values('ChirpNum')
    points = var_data.index.values
    
    flags = np.logical_or(var_data == np.inf, var_data == -np.inf)
    flags = np.logical_or(flags, var_data == np.nan)
    
    point_ind, chirp_ind = np.where(flags)
    bad_chirps = chirps[chirp_ind]
    
    bad_list = np.array([(x,y) for x,y in zip(point_ind,bad_chirps)])
    
    bad_dict[var] = bad_list
    

for var in stack:
    bad_list = bad_dict[var]
    xvals = samp.gridmeasurements[meas].GetDataSubset(stack=stack, inout=inout).columns.get_level_values('xaxis').astype(float)
    chirps = samp.gridmeasurements[meas].GetDataSubset(stack=stack, inout=inout).columns.get_level_values('ChirpNum').astype(float)

    if bad_list.size != 0:
        f = plt.figure(figsize=(8,8))

        gs = gridspec.GridSpec(2, 1,hspace=0, height_ratios=[1,0.2])

        ax1 = plt.subplot(gs[0])
        ax1.scatter(bad_list[:,1],bad_list[:,0])
        ax1.set_ylabel('Point')
        plt.title(str(stack)+ ' Flagged Fits')

        ax2 = plt.subplot(gs[1])
        ax2.plot(chirps,xvals)
        ax2.plot(chirps,np.full(len(chirps),0),'k--',linewidth=0.8)
        ax2.set_ylabel('Voltage (V)')
        ax2.set_xlabel('Chirp Num')
